In [1]:
import numpy as np
import h5py

import seaborn as sns
import matplotlib.pyplot as plt

import pandas as pd

ModuleNotFoundError: No module named 'h5py'

In [ ]:
def get_keys(file_name):
    with h5py.File(file_name, 'r') as f:
        keys = []
        for k in f.keys():
            keys.append(k)
        return keys

def get_time(file_name):
    with h5py.File(file_name, 'r') as f:
        dt = f["dt"][()]
        tmax = f["t_max"][()]
        return dt, tmax
    
def get_seeds(file_name):
    with h5py.File(file_name, 'r') as f:
        seeds = f["seeds"][()]
        return seeds
    
def get_noise_levels(file_name, state):
    with h5py.File(file_name, 'r') as f:
        noise_levels = []
        for k in f[str(state)].keys():
            noise_levels.append(k)
        return noise_levels

def get_by_name(file_name, states, var_name, noise_levels):
    var_by_name = []
    with h5py.File(file_name, 'r') as f:
        for w in noise_levels:
            var_by_name.append([ f[str(state)][str(w)][str(var_name)][()] for state in states])
        return(var_by_name)
    
def get_variable_names(file_name, state, noise_level):
    with h5py.File(file_name, 'r') as f:
        var_names = []
        for k in f[str(state)][str(noise_level)].keys():
            var_names.append(k)
        return(var_names)

In [ ]:
file_name = "C:/Users/Zakhar/Documents/GitHub/JPOP_SID/Kurt_clust_compare_methods_started_2022-Jul-18_at_13-06.h5"
#get_keys(file_name)

In [ ]:
noise_levels = get_noise_levels(file_name, 'State_B1')
get_variable_names(file_name, 'State_B1', noise_levels[1])
basis_states = [ 'State_B' + str(_) for _ in range(1,5)]
dodeca_states = ['State_D' + str(_) for _ in range(1,21)]
all_states = [*basis_states, *dodeca_states]

In [ ]:
fidelity_lsid = get_by_name(file_name, all_states, "fidelity_lsid", noise_levels)
fidelity_kraus = get_by_name(file_name, all_states, "fidelity_kraus", noise_levels)
fidelity_pade = get_by_name(file_name, all_states, "fidelity_pade", noise_levels)
fidelity_simp = get_by_name(file_name, all_states, "fidelity_simp", noise_levels)

In [ ]:
print("Pade: ", sum(len(_) for _ in fidelity_pade))

print(
        sum(len(_) for _ in fidelity_pade)
        +
        sum(len(_) for _ in fidelity_kraus)
        +
        sum(len(_) for _ in fidelity_simp)
)

convergence_pade = [100. * np.isfinite(_).sum() / len(_) for _ in fidelity_pade]
convergence_kraus = [100. * np.isfinite(_).sum() / len(_) for _ in fidelity_kraus]
convergence_simps = [100. * np.isfinite(_).sum() / len(_) for _ in fidelity_simp]
convergence_lsid = [100. * np.isfinite(_).sum() / len(_) for _ in fidelity_lsid]

x = np.arange(len(noise_levels))  # the label locations
width = 0.3  # the width of the bars

fig, ax = plt.subplots()

rects = [
        ax.bar(x - width, convergence_pade, width, label='Pade'),
        ax.bar(x, convergence_simps, width, label='Simpson'),
        ax.bar(x + width, convergence_kraus, width, label='Kraus'),
]

ax.set_ylabel('Percentile of reconstructions converged')
ax.set_xlabel("Noise (w)")
ax.set_xticks(x, noise_levels)
ax.legend()

fig.tight_layout()

plt.show()



In [ ]:
def get_DataFrame(data):
        frame = {k: np.hstack(v) for k, v in data.items()}
        frame['noise ($w$)'] = np.hstack(
                [np.repeat(w, len(d)) for w, d in zip(noise_levels, list(data.values())[0])]
        )
        return frame

fig, ax = plt.subplots(1, 3, sharey=True, sharex=True, figsize=(3 * 3, 4), dpi=300,)

params = dict(
        x='Minimal fidelity',
        hue='noise ($w$)',
        fill=True,
        common_norm=False,
        log_scale=(False, True),
)

# Pade objective function
#sns.kdeplot(
#        data=get_DataFrame({'Minimal fidelity': fidelity_pade}), ax=ax[0], **params
#)

# Linear objective function
sns.kdeplot(
        data=get_DataFrame({'Minimal fidelity': fidelity_lsid}), ax=ax[0], **params
)

# Kraus objective function
sns.kdeplot(
        data=get_DataFrame({'Minimal fidelity': fidelity_kraus}), ax=ax[1], **params
)

# integral Simpson objective function
sns.kdeplot(
        data=get_DataFrame({'Minimal fidelity': fidelity_simp}), ax=ax[2], **params
)

#label = list("bcd")[::-1]

label = ["Linear", "Kraus", "Simpson"]

for _ in ax:
        sns.move_legend(_, "upper left")
        _.set_xlabel(r'$F_{\rm min}$')
        _.text(0.1, 0.35, "({})".format(label.pop()), transform=_.transAxes)

ax[0].set_ylabel('Number of tests')

plt.ylim([1, 1e5])
plt.xlim([0.98, 1.0005])
plt.tight_layout()
plt.savefig("Kurt_LiPoSID_fidelity.pdf")

plt.show()


In [ ]:
from scipy.stats import pearsonr
correlations = [pearsonr(*_)[0] for _ in zip(fidelity_pade, fidelity_kraus)]
sns.barplot(noise_levels, correlations)
plt.xlabel('noise ($w$)')
plt.ylabel('Correlation between Pade and Kraus min fieldity')
plt.show()



In [ ]:
plt.violin(fidelity_pade)